In [96]:
import cv2 #pip install opencv-python
import pandas as pd #pip install pandas
import os #no need to install
from pyzbar.pyzbar import decode #pip install pyzbar 
from openpyxl import load_workbook #pip install openpyxl
import numpy as np
os.chdir("D:\LEARN\Python\Final_Project")

### Ex3: Create a student dataset with ID

In [19]:
student_data = pd.read_excel("DATA.xlsx", sheet_name = "Student data")
student_data.head()

,Student_ID,STUDENT,Student_Major,Student_Class,Student_Email
0,20040004,Nguyễn Quang Hào,DS,20DS,hao.nguyen200404@vnuk.edu.vn
1,20050005,Phan Thanh Lâm,THM,20THM2,NaN
2,20010039,Lê Đại Hải,IBM,20IBM3,NaN
3,20040000,Lê Thị Tấm,CSE,20CSE,NaN
4,20040001,Trần Chín,THM,20THM1,NaN


### Ex4: Create a book dataset with code 

In [20]:
book_data = pd.read_excel("DATA.xlsx", sheet_name = "Book data")
book_data['CODE'] = book_data['CODE'].apply(lambda x: '{:.0f}'.format(x))
book_data.head()

,CODE,BOOK,AUTHOR
0,8936037799377,Những Cuộc Phiêu Lưu Trong Kinh Doanh,John Brooks
1,8935251405972,Tư Duy Nhanh Và Chậm,Daniel Kahneman
2,8935235219489,Vào Trong Hoang Dã,Jon Krakauer
3,8935235229693,Tương Lai Sau Đại Dịch Covid,Jason Schenker
4,8935235218130,Yêu những điều không hoàn hảo,Hea Min


### Ex5: Scan infomation on Student Card

In [25]:
card = cv2.imread('Student_Card/STUDENT_CARD_00.jpg')

def student_card_detect(card):
    code = decode(card)
    id = code[0].data.decode('utf-8')
    for i in range(len(student_data)):
        if int(id) == int(student_data.loc[i,'Student_ID']):
            return(student_data.loc[i])

print(student_card_detect(card)) 


Student_ID                           20040004
STUDENT                      Nguyễn Quang Hào
Student_Major                              DS
Student_Class                            20DS
Student_Email    hao.nguyen200404@vnuk.edu.vn
Name: 0, dtype: object
Y


### Ex6: Get information on Book

In [88]:
book_01 = cv2.imread("Book_List/BOOK_01.jpg")
book_02 = cv2.imread("Book_List/BOOK_02.jpg")
book_03 = cv2.imread("Book_List/BOOK_03.jpg")

def book_detect(book):
    book_code = decode(book)
    code = book_code[0].data.decode('utf-8')

    for i in range(len(book_data)):
        if code == book_data.loc[i,'CODE']:
            return(book_data.loc[i])
            
# print(book_detect(book_02))
# print(book_info['BOOK'])

CODE           8935235219489
BOOK      Vào Trong Hoang Dã
AUTHOR          Jon Krakauer
Name: 2, dtype: object


In [115]:
card_info = student_card_detect(card)
book_info = book_detect(book_01)
book_info_02 = book_detect(book_02)
book_info_03 = book_detect(book_03)
borrow_list = pd.DataFrame(columns=['BOOK','ID','Student_name'])

def borrow_book(card_info, book_info):
    borrow_info = [[book_info['BOOK'],card_info['Student_ID'],card_info['STUDENT']]]
    # borrow_info = pd.DataFrame({'BOOK':book_info['BOOK'],'ID':card_info['Student_ID'],'Student_name':card_info['STUDENT']}, index = [0])
    return borrow_info

def borrow_list(borrow_book):
    global borrow_list
    borrow_list = borrow_list.append(borrow_book,ignore_index = True)
    return borrow_list
borrow_book_01 = borrow_book(card_info, book_info)
borrow_list(borrow_book_01)
# borrow_list = pd.DataFrame(borrow_book_01,columns=['BOOK','ID','Student_name'])
# borrow_list = borrow_list.append(borrow_book_01,ignore_index = True)
# borrow_book_02 = borrow_book(card_info, book_info_02)
# borrow_list = borrow_list.append(borrow_book_02,ignore_index = True)
# borrow_book_03 = borrow_book(card_info, book_info_03)
# borrow_list = borrow_list.append(borrow_book_03,ignore_index = True)
    # return borrow_list
# borrow_1 = borrow_book(card_info,book_detect(book_01))
# borrow_list = borrow_list(borrow_list,borrow_1)

# file = pd.ExcelWriter('DATA.xlsx', mode='a', if_sheet_exists = 'replace')
# borrow_list.to_excel(file, 'Borrowed data')
# file.save()
print(borrow_list)

AttributeError: 'function' object has no attribute 'append'